# Step 3 - Feature Engineering

Normalize features to ensure they contribute equally to the model. This process can help improve the performance and convergence of various machine learning algorithms.

After this, perform data augmentation using SMOTE technique.

In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

## 3.0 - Load the Data

In [5]:
previous_years_df = pd.read_csv('../data/02_previous_years_not_null.csv')
present_year_df = pd.read_csv('../data/04_present_year_not_null.csv')


## 3.1 - Scaling the Data

MinMaxScaler(feature_range = (0, 1)) will transform each value in the column proportionally within the range [0,1]. We use this as the scaler choice to transform the feature and preserve the shape of the dataset (no distortion).

Besides, we will also encode the class 'neg' as 0 and class 'pos' as 1 for further use.


In [6]:
# Function to scale the data
def scaling(data):
    # Separate the target and feature columns
    X = data.drop('class', axis=1)  # Features
    y = data['class']               # Target

    # Initialize the MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler.fit_transform(X)

    # Convert the scaled features back to a DataFrame
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
    scaled_data = pd.concat([X_scaled_df, y.reset_index(drop=True)], axis=1)

    scaled_data['class'] = scaled_data['class'].map({'neg': 0, 'pos': 1})

    return scaled_data

**Previous Years**

In [7]:
previous_scaled = scaling(previous_years_df)
previous_scaled.head(5)

,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000,class
0,0.027925,0.0,9.999998e-01,3.261769e-08,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.013067,7.421784e-03,8.179508e-03,1.073013e-02,0.001321,0.003800,0.000000,0.000000,0.000000,0
1,0.012036,0.0,0.000000e+00,1.467796e-08,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.004716,3.019031e-03,4.272917e-03,4.228511e-03,0.000679,0.005064,0.000394,0.000000,0.000000,0
2,0.014942,0.0,1.070067e-07,1.164918e-08,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.004232,4.364196e-03,7.130884e-03,1.014768e-02,0.001321,0.004937,0.000135,0.000000,0.000000,0
3,0.000004,0.0,3.285295e-08,7.688457e-09,0.0,0.000498,0.0,0.0,0.0,0.000005,...,0.000001,5.970003e-07,7.660803e-07,3.163775e-07,0.000000,0.000000,0.000000,0.008299,0.027923,0
4,0.022164,0.0,6.420405e-07,5.335323e-08,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.006086,4.171776e-03,6.044861e-03,9.078578e-03,0.002605,0.022523,0.000320,0.000000,0.000000,0


**Present Year:**

In [8]:
present_scaled = scaling(present_year_df)
present_scaled.head(5)

,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000,class
0,0.000001,0.000000,9.386558e-09,0.000138,0.0,0.0,0.0,0.0,0.0,0.000037,...,0.000006,0.000008,0.000012,0.000002,0.000002,0.000000,0.000000,0.0,0.0,0
1,0.000002,0.000000,3.191430e-08,0.000460,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000013,0.000033,0.000002,0.000002,0.000010,0.000000,0.000000,0.0,0.0,0
2,0.001537,0.034483,9.949751e-08,0.001287,0.0,0.0,0.0,0.0,0.0,0.002731,...,0.017227,0.009057,0.004810,0.031192,0.003237,0.000030,0.000000,0.0,0.0,0
3,0.001393,0.000000,4.740212e-07,0.010754,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.011018,0.009945,0.008660,0.010232,0.004425,0.016389,0.000707,0.0,0.0,0
4,0.000042,0.000000,7.321515e-08,0.001608,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000188,0.000380,0.000888,0.000075,0.000010,0.000004,0.000000,0.0,0.0,0


## 3.2 - Saving Original Data Scaled

In [9]:
previous_scaled.to_csv('../data/05_previous_years_scaled.csv', index=False)
present_scaled.to_csv('../data/06_present_year_scaled.csv', index=False)

## 3.3 - Data Augmentation

Apply SMOTE (Synthetic Minority Over-sampling Technique) to balance the class distribution by generating synthetic samples for the minority class.

It should be applied only to the training data to prevent data leakage and ensure that the test data remains representative of the real-world scenario. It will be explain in detaild in the 5th notebook that the training data here is the Previous Years data.

In [17]:
# Separate target and features for training data
y_train = previous_scaled['class']
X_train = previous_scaled.drop('class', axis=1)

# Apply SMOTE to training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Count the occurrences of each class in the resampled target variable
class_counts = pd.Series(y_train_resampled).value_counts()

print("Training Resampled features shape:", X_train_resampled.shape)
print("Training Resampled target shape:", y_train_resampled.shape)
print(f"\nClass Count: {class_counts}")

Training Augmented features shape: (118000, 170)
Training Augmented target shape: (118000,)

Class Count: class
0    59000
1    59000
Name: count, dtype: int64


## 3.4 - Saving Resampled (Augmented) Data

In [20]:
# Create a DataFrame from the resampled data
X_train_resampled_df = pd.DataFrame(X_train_resampled, columns=X_train.columns)
y_train_resampled_df = pd.DataFrame(y_train_resampled, columns=['class'])

# Combine features and target into a single DataFrame
SMOTEd_df = pd.concat([X_train_resampled_df, y_train_resampled_df], axis=1)

In [21]:
SMOTEd_df.to_csv('../data/07_previous_years_SMOTEd.csv', index=False)